You must run this notebook on a GPU. A T4 is sufficient. It's free on [Google
Colab](https://stackoverflow.com/questions/62596466/how-can-i-run-notebooks-of-a-github-project-in-google-colab/67344477#67344477).

**Description**: for a [GPTQd Mistral
7B](https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GPTQ) and the [Craigslist
Bargains](https://huggingface.co/datasets/craigslist_bargains) classification task, this
notebook demonstrates that CAPPr gets you +1% absolute accuracy compared to text
generation. In general, you should expect similar or identical performance when every
completion is 1 token long. This notebook also demonstrates using the `prior` keyword argument.

**Estimated run time**: ~15 min.

In [1]:
# check correct CUDA version
import torch

_cuda_version = torch.version.cuda
_msg = (
    "Change the pip install auto-gptq command to the one for "
    f"{_cuda_version} based on the list here: "
    "https://github.com/PanQiWei/AutoGPTQ#quick-installation"
)

assert _cuda_version == "11.8", _msg

In [ ]:
!python -m pip install "cappr[demos] @ git+https://github.com/kddubey/cappr.git" \
auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ \
optimum

In [3]:
from __future__ import annotations
from typing import Sequence

import datasets
import numpy as np
import pandas as pd
import torch
from tqdm.auto import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

from cappr.huggingface.classify import token_logprobs, predict_proba

# Load training data

If you're only interested in how to use CAPPr, skip this section.

In [ ]:
_df_raw_tr = pd.DataFrame(datasets.load_dataset("craigslist_bargains", split="train"))

In [5]:
len(_df_raw_tr)

5247

The "text" to classify is gonna be the Buyer-Seller dialogue. Need to process that into
something an LLM would better understand.

In [6]:
_df_raw_tr["utterance"][0]  # see those last two empty strings. gonna drop em

['Hi, not sure if the charger would work for my car. Can you sell it to me for $5?',
 'It will work, i have never seen a car without a cigarette lighter port.\\',
 "Still, can I buy it for $5? I'm on a tight budge",
 'I think the lowest I would want to go is 8. ',
 "How about $6 and I pick it up myself? It'll save you shipping to me.",
 '7, and we have a deal.',
 'Eh, fine. $7.',
 '',
 '']

In [7]:
assert (_df_raw_tr["agent_turn"].apply(len) == _df_raw_tr["utterance"].apply(len)).all()

The possible choices for each dialogue are product categories:

In [8]:
_df_raw_tr["items"].apply(lambda item: item["Category"])

0               [phone, phone]
1                 [bike, bike]
2           [housing, housing]
3       [furniture, furniture]
4       [furniture, furniture]
                 ...          
5242                [car, car]
5243    [furniture, furniture]
5244              [bike, bike]
5245    [furniture, furniture]
5246        [housing, housing]
Name: items, Length: 5247, dtype: object

Not sure why they're duplicated. Are they ever different? Hope not.

In [9]:
assert (_df_raw_tr["items"].apply(lambda item: len(set(item["Category"]))) == 1).all()

In [10]:
class_names = sorted(set(_df_raw_tr["items"].apply(lambda item: item["Category"][0])))
class_names

['bike', 'car', 'electronics', 'furniture', 'housing', 'phone']

I'm gonna assume this is the complete list.

# Write prompt

Instruction format pulled from
[here](https://huggingface.co/TheBloke/Mistral-7B-OpenOrca-GPTQ#you-can-then-use-the-following-code).

In [11]:
chat_template = """
<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
""".lstrip("\n")

In [12]:
class_names_str = ", ".join(class_names)

In [13]:
def prompt(dialogue: str) -> str:
    system_message = (
        "You will be given a dialogue between a seller and a buyer about the sale of "
        "an item.\n"
        "Your task is to categorize the item being sold as one of these categories: "
        f"{class_names_str}\n\n"
        "You will answer with just the the correct category and nothing else."
    )
    user_message = (
        "Dialogue:\n"
        f'"""\n{dialogue}\n"""\n'
        "The item discussed in the dialogue above belongs to the category:"
    )
    return chat_template.format(
        system_message=system_message, prompt=user_message
    )

# Process training data

If you're only interested in how to use CAPPr, skip this section.

To ensure train and test data processing is equivalent, we'll apply the same function:
`process`.

In [14]:
def _as_dialogue(
    all_agent_turns: Sequence[Sequence[bool]], all_utterances: Sequence[Sequence[str]]
):
    if len(all_agent_turns) != len(all_utterances):
        raise ValueError("agent_turns and utterances must have the same length.")

    dialogues = []
    for agent_turns, utterances in zip(all_agent_turns, all_utterances):
        dialogue: list[str] = []
        for agent_turn, utterance in zip(agent_turns, utterances):
            if not utterance:
                # some utterances are empty for some reason. just gonna drop em
                continue
            prefix = "Buyer: " if not agent_turn else "Seller: "
            dialogue.append(prefix + utterance)
        dialogues.append("\n".join(dialogue))
    return dialogues


def process(craigslist_df: pd.DataFrame) -> pd.DataFrame:
    """
    Returns a DataFrame with the canonical `"text"`, `"label"`, and `"prompt"` columns
    for the
    [CraigslistBargains dataset](https://huggingface.co/datasets/craigslist_bargains),
    assuming it was loaded via::

        craigslist_df = pd.DataFrame(
            datasets.load_dataset("craigslist_bargains", split=...)
        )
    """
    # Input checks
    if not (
        craigslist_df["agent_turn"].apply(len) == craigslist_df["utterance"].apply(len)
    ).all():
        raise ValueError("There's an agent_turn and utterance with different lengths.")
    if not (
        craigslist_df["items"].apply(lambda item: len(set(item["Category"]))) == 1
    ).all():
        raise ValueError("There's an item associated with multiple categories.")

    class_names = ["bike", "car", "electronics", "furniture", "housing", "phone"]
    # hard-coded per dataset to ensure consistency across splits. it's possible that the
    # test split is missing some of these.
    df = pd.DataFrame(
        {
            "text": _as_dialogue(
                craigslist_df["agent_turn"], craigslist_df["utterance"]
            ),
            "class": [item["Category"][0] for item in craigslist_df["items"]],
        },
        index=craigslist_df.index,
    )
    assert len(df) == len(craigslist_df)
    df["label"] = [class_names.index(class_name) for class_name in df["class"]]
    df["prompt"] = [prompt(dialogue) for dialogue in df["text"]]
    return df[["text", "label", "prompt", "class"]]

In [15]:
df_tr = process(craigslist_df=_df_raw_tr)

In [16]:
len(df_tr)

5247

For computationally and statistically cheap experiments, we don't need all of this data.

In [17]:
df_tr_mini = df_tr.copy().sample(n=100, random_state=123).reset_index(drop=True)
len(df_tr_mini)

100

# Preview prompt

In [18]:
print(df_tr["prompt"].iloc[0])

<|im_start|>system
You will be given a dialogue between a seller and a buyer about the sale of an item.
Your task is to categorize the item being sold as one of these categories: bike, car, electronics, furniture, housing, phone

You will answer with just the the correct category and nothing else.<|im_end|>
<|im_start|>user
Dialogue:
"""
Buyer: Hi, not sure if the charger would work for my car. Can you sell it to me for $5?
Seller: It will work, i have never seen a car without a cigarette lighter port.\
Buyer: Still, can I buy it for $5? I'm on a tight budge
Seller: I think the lowest I would want to go is 8. 
Buyer: How about $6 and I pick it up myself? It'll save you shipping to me.
Seller: 7, and we have a deal.
Buyer: Eh, fine. $7.
"""
The item discussed in the dialogue above belongs to the category:<|im_end|>
<|im_start|>assistant



In [19]:
print(df_tr["class"].iloc[0])

phone


Ooh, little tricky since the buyer and seller only mention a charger. I bet most models
will incorrectly classify the dialogue as a car sale.

# Load model

In [ ]:
model_name = "TheBloke/Mistral-7B-OpenOrca-GPTQ"
model = AutoModelForCausalLM.from_pretrained(
    model_name, device_map="auto", trust_remote_code=False, revision="main"
)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [21]:
_ = model(**tokenizer(["warm up"], return_tensors="pt").to(model.device))

# CAPPr

Many research datasets are intentionally balanced (so that only the model's likelihood is considered).
Balanced classes in real applications are rare. Let's see what the distribution
looks like for this (more realistic) dataset:

In [22]:
df_tr["class"].value_counts(normalize=True).sort_index()

bike           0.179150
car            0.133028
electronics    0.130741
furniture      0.247951
housing        0.202783
phone          0.106346
Name: class, dtype: float64

Indeed, not really balanced. Let's supply this prior to CAPPr and see how it does on a
small subset of the training data.

In [23]:
prior = (
    df_tr["class"]
    .value_counts(normalize=True)
    [class_names]
)

In [24]:
pred_probs = predict_proba(
    prompts=df_tr_mini["prompt"],
    completions=class_names,
    model_and_tokenizer=(model, tokenizer),
    prior=prior,
)

conditional log-probs:   0%|          | 0/100 [00:00<?, ?it/s]

Accuracy:

In [25]:
(pred_probs.argmax(axis=1) == df_tr_mini["label"]).mean()

0.92

# Text generation

We need to create a PyTorch Dataset to batch the inputs.

In [26]:
class TextsDataset(torch.utils.data.Dataset):
    def __init__(self, texts: list[str]):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index: int):
        return self.texts[index]

We'll do greedy sampling to better ensure that each completion is one of the class names.

In [27]:
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=5,
    return_full_text=False,
)
# pad to allow batching. these get masked out ofc
generator.tokenizer.pad_token_id = generator.model.config.eos_token_id

In [28]:
text_gen_dataset_tr = TextsDataset(df_tr_mini["prompt"].tolist())

In [29]:
completions = []
for seq in tqdm(
    generator(
        text_gen_dataset_tr,
        # suppress "Setting pad_token_id..." stdout
        pad_token_id=generator.tokenizer.eos_token_id,
        batch_size=2,
    ),
    total=len(text_gen_dataset_tr),
    desc="Sampling",
):
    completions.append(seq[0]['generated_text'])

Sampling:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Let's see if the model generated categories like we asked.

In [30]:
pd.Series(completions).sample(n=10)

56     electronics
27             car
84         housing
59     electronics
77             car
89         housing
58            bike
1      electronics
65     electronics
91     electronics
dtype: object

Nice, text generation works well here. Writing the post-processor is trivial.

In [31]:
def process_completion(completion: str, class_names: Sequence[str], default=-1) -> int:
    for i, name in enumerate(class_names):
        if name in completion.lower():
            return i
    return default

In [32]:
preds_text_gen = [
    process_completion(completion, class_names)
    for completion in completions
]

How many of the completions could be mapped to a label?

In [33]:
(pd.Series(preds_text_gen) != -1).mean()

1.0

How accurate are the predictions?

In [34]:
(df_tr_mini["label"] == preds_text_gen).mean()

0.91

# Evaluate on test data

It's important to do some things honorably.

<span style="font-family: Baskerville; font-size: 18px;">I solemnly swear that I
evaluated on the test set twice (once per pre-selected method), running only the
following cells in sequence once.</span>

<img src="../signature.png" alt="drawing" width="200"/>
<div style="width:200px"><hr/></div>

In [35]:
_df_raw_te = pd.DataFrame(datasets.load_dataset("craigslist_bargains", split="test"))
len(_df_raw_te)

838

In [36]:
df_te = process(craigslist_df=_df_raw_te)

## Text generation

In [37]:
text_gen_dataset_te = TextsDataset(df_te["prompt"].tolist())

In [38]:
completions_te = []
for seq in tqdm(
    generator(
        text_gen_dataset_te,
        # suppress "Setting pad_token_id..." stdout
        pad_token_id=generator.tokenizer.eos_token_id,
        batch_size=2,
    ),
    total=len(text_gen_dataset_te),
    desc="Sampling",
):
    completions_te.append(seq[0]['generated_text'])

Sampling:   0%|          | 0/838 [00:00<?, ?it/s]

In [39]:
preds_text_gen_te = [
    process_completion(completion, class_names)
    for completion in completions_te
]

How many of the completions could be mapped to a label?

In [40]:
(pd.Series(preds_text_gen_te) != -1).mean()

0.9964200477326969

How accurate are the predictions?

In [41]:
(preds_text_gen_te == df_te["label"]).mean()

0.8937947494033412

## CAPPr

In [42]:
pred_probs_te = predict_proba(
    prompts=df_te["prompt"],
    completions=class_names,
    model_and_tokenizer=(model, tokenizer),
    batch_size=2,
    prior=prior,  # estimated from independent training data
)

conditional log-probs:   0%|          | 0/838 [00:00<?, ?it/s]

How accurate are the predictions?

In [43]:
(pred_probs_te.argmax(axis=1) == df_te["label"]).mean()

0.9045346062052506

Hmm, [Refuel AI's few-shot
experiment](https://github.com/refuel-ai/autolabel/blob/main/examples/craigslist/example_craigslist.ipynb)
with `gpt-3.5-turbo` got 89% accuracy. But the way that experiment processed dialogues
seems off. So I'm not gonna say this 4 GB model beats `gpt-3.5-turbo` just yet.